In [50]:
import pandas as pd
import numpy as np
import collections
import tensorflow as tf
import gensim

def createNeighbor(df):
    data = []
    for each_line in df.iterrows():
        for header in ["Proto","SrcAddr","Dport"]:
            if header == "SrcAddr":
                data.append([each_line[1][header],each_line[1]["DstAddr"]])
                data.append([each_line[1][header],each_line[1]["Proto"]])
                data.append([each_line[1][header],str(each_line[1]["Dport"])])
            else:
                data.append([str(each_line[1][header]),each_line[1]["DstAddr"]])

    return data

df = pd.read_csv("corpus_example.csv")

word2int={}
sentences = []
wordlist = []
for each_line in df.iterrows():
    sentences.append(each_line[1].tolist())
    for header in ["Proto","SrcAddr","DstAddr","Dport"]:
        wordlist.append(str(each_line[1][header]))

myCounter = collections.Counter(wordlist)
print(myCounter.most_common())
wordlist=set(wordlist)

neighbor = createNeighbor(df)
hi = pd.DataFrame(neighbor, columns = ['input', 'label'])

for i,word in enumerate(wordlist):
    word2int[word] = i

#for each in word2int.keys():
#    print(each+":"+str(word2int[each]))




[('tcp', 270), ('udp', 220), ('147.32.84.229', 186), ('13363', 88), ('147.32.80.9', 73), ('53', 73), ('80', 70), ('147.32.84.59', 59), ('443', 43), ('147.32.84.118', 41), ('6881', 39), ('147.32.85.25', 39), ('140.115.25.74', 30), ('147.32.84.138', 29), ('147.32.86.165', 25), ('173.242.220.227', 17), ('147.32.84.111', 17), ('188.121.222.1', 12), ('147.32.84.2', 11), ('147.32.3.51', 10), ('10010', 10), ('43087', 8), ('122.174.15.39', 7), ('12114', 7), ('147.32.87.22', 6), ('77.78.99.22', 6), ('147.32.86.24', 6), ('91.207.59.162', 6), ('203.23.120.136', 5), ('147.32.84.46', 4), ('147.32.86.166', 4), ('147.32.85.56', 4), ('147.32.86.223', 4), ('74.125.232.213', 4), ('74.125.232.220', 4), ('88.212.37.169', 3), ('94.44.60.103', 3), ('188.112.125.201', 3), ('212.201.86.130', 3), ('92.40.253.219', 3), ('147.32.80.13', 3), ('147.32.86.110', 3), ('8080', 3), ('213.155.227.215', 3), ('74.125.232.215', 3), ('161', 3), ('123', 3), ('147.32.85.18', 3), ('2.159.127.100', 2), ('85.132.162.9', 2), ('14

In [51]:
df.head(5)

,Proto,SrcAddr,DstAddr,Dport
0,tcp,93.45.239.29,147.32.84.118,6881
1,tcp,62.240.166.118,147.32.84.229,13363
2,tcp,147.32.86.148,66.235.132.232,80
3,tcp,147.32.3.51,147.32.84.46,10010
4,tcp,88.212.37.169,147.32.84.118,6881


In [60]:
hi.head(5)

,input,label
0,tcp,147.32.84.118
1,93.45.239.29,147.32.84.118
2,93.45.239.29,tcp
3,93.45.239.29,6881
4,6881,147.32.84.118


In [61]:
ONE_HOT_DIM = len(wordlist)

# function to convert numbers to one hot vectors
def to_one_hot_encoding(data_point_index):
    one_hot_encoding = np.zeros(ONE_HOT_DIM)
    one_hot_encoding[data_point_index] = 1
    return one_hot_encoding

X = [] # input word
Y = [] # target word

for x, y in zip(hi['input'], hi['label']):
    X.append(to_one_hot_encoding(word2int[ x ]))
    Y.append(to_one_hot_encoding(word2int[ y ]))

# convert them to numpy arrays
X_train = np.asarray(X)
Y_train = np.asarray(Y)

# making placeholders for X_train and Y_train
x = tf.placeholder(tf.float32, shape=(None, ONE_HOT_DIM))
y_label = tf.placeholder(tf.float32, shape=(None, ONE_HOT_DIM))

# word embedding will be 2 dimension for 2d visualization
EMBEDDING_DIM = 100

# hidden layer: which represents word vector eventually
W1 = tf.Variable(tf.random_normal([ONE_HOT_DIM, EMBEDDING_DIM]))
b1 = tf.Variable(tf.random_normal([1])) #bias
hidden_layer = tf.add(tf.matmul(x,W1), b1)

# output layer
W2 = tf.Variable(tf.random_normal([EMBEDDING_DIM, ONE_HOT_DIM]))
b2 = tf.Variable(tf.random_normal([1]))
prediction = tf.nn.softmax(tf.add( tf.matmul(hidden_layer, W2), b2))

# loss function: cross entropy
loss = tf.reduce_mean(-tf.reduce_sum(y_label * tf.log(prediction), axis=[1]))

# training operation
train_op = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

In [66]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init) 

iteration = 2000
for i in range(iteration):
    # input is X_train which is one hot encoded word
    # label is Y_train which is one hot encoded neighbor word
    sess.run(train_op, feed_dict={x: X_train, y_label: Y_train})
    if i % 10 == 0:
        print('iteration '+str(i)+' loss is : ', sess.run(loss, feed_dict={x: X_train, y_label: Y_train}))

iteration 0 loss is :  nan


KeyboardInterrupt: 

In [ ]:
vectors = sess.run(W1 + b1)
print(vectors)

In [ ]:
w2v_df = pd.DataFrame(vectors, columns = ["x"+str(i) for i in range(1,101) ])
w2v_df['word'] = wordlist
head_list = ["x"+str(i) for i in range(1,101)]
w2v_df = w2v_df[['word']+head_list]
w2v_df

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

"Creates and TSNE model and plots it"
labels = []
tokens = []

    
tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
new_values = tsne_model.fit_transform(vectors)

x = []
y = []
for value in new_values:
    x.append(value[0])
    y.append(value[1])
        
plt.figure(figsize=(16, 16)) 
for i in range(len(x)):
    plt.scatter(x[i],y[i],marker='o',c='blue')

plt.show()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

k_cluster = pd.DataFrame(new_value,columns=['x', 'y'])

# visualize data point
sns.lmplot('x', 'y', data=k_cluster, fit_reg=False, scatter_kws={"s": 200}) # x-axis, y-axis, data, no line, marker size

# title
plt.title('kmean plot')

# x-axis label
plt.xlabel('x')
# y-axis label
plt.ylabel('y')
# convert dataframe to numpy array
data_points = k_cluster.values
kmeans = KMeans(n_clusters=3).fit(data_points)
kmeans.labels_
kmeans.cluster_centers_
k_cluster['cluster_id'] = kmeans.labels_
k_cluster.head(12)


In [ ]:
sns.lmplot('x', 'y', data=k_cluster, fit_reg=False,  # x-axis, y-axis, data, no line
           scatter_kws={"s": 150}, # marker size
           hue="cluster_id") # color

# title
plt.title('after kmean clustering')